<center><font size=5>可转债因子分析</font></center>

使用机器学习代替IC来进行初筛。由于IC无法反映因子在机器学习中的效果。

加载各种包

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append('..')  # 文件位于

import factors  # 因子计算
import data_utils_CB  # 数据读取
import performance as perf  # IC计算
import plotting  #绘图

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 1. 初始化DataLoader 

DataLoader实现了对csv文件的批量读取，会按照\[年\]\[月\]保存在dl中。
一些重要函数：
- init_month_df: 加载目标文件夹下的所有zip文件。这些文件读取后会被保存在dataloader中。注意，

In [2]:
dl = data_utils_CB.DataLoader()
dl.init_month_df('E:/jupyternote/QF/cb/raw_data/2023_raw/')
# 计算未来收益率
data_utils_CB.compute_future_rtn_for_all(dl)

# 2. 计算因子
ta Core indicators
测试批量生成因子

In [3]:
import pandas_ta as ta
import pandas as pd
from tqdm import tqdm
import time

e = pd.DataFrame()
core_indicators = e.ta.indicators(as_list=True)  # 获取所有indicators列表

In [4]:
core_indicators.remove('above')
core_indicators.remove('above_value')
core_indicators.remove('cross_value')
core_indicators.remove('aberration')
core_indicators.remove('below')
core_indicators.remove('below_value')

### indicators函数有效性的检验
1. 计算不能报错
2. 计算时间不能太长

In [5]:
test_df = dl.df_dict['110052.csv'].copy()

In [6]:
def preprocess(df):
    df.rename({'vol':'volume'},axis=1, inplace=True)
    return df

In [7]:
test_df = preprocess(test_df)
# 遍历判断哪些是直接可用的
remove_fun_list = []  # 不需要计算的func
time_list = []  # 保存函数计算的时长
for indicator_name in tqdm(core_indicators):
    try:
        fun = getattr(test_df.ta, indicator_name)
        # 记录代码开始时间
        start_time = time.time()
        for i in range(1):
            temp_data = fun(append=True)
        # 记录代码结束时间
        end_time = time.time()
        # 计算代码执行时间
        execution_time = end_time - start_time
        if type(temp_data) == pd.core.frame.DataFrame and 'open_time' in temp_data.columns:
            remove_fun_list.append(indicator_name)
            time_list.append(999)
        else:
            time_list.append(execution_time)
    except:
        remove_fun_list.append(indicator_name)
        time_list.append(999)

100%|████████████████████████████████████████████████████████████████████████████████| 137/137 [00:58<00:00,  2.34it/s]

[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.


In [8]:
time_df = pd.DataFrame({
    'factor': core_indicators,
    'time':time_list
})

In [9]:
time_df.shape

(137, 2)

In [10]:
time_df.sort_values('time').tail(10)

,factor,time
126,vidya,1.709454
4,alma,1.753654
42,ha,1.870057
109,supertrend,2.023046
65,mcgd,3.072467
43,hilo,3.194684
79,psar,3.866484
87,qqe,7.254401
112,td_seq,25.499077
129,vwap,999.000000


In [11]:
valid_core_indicators = list(time_df.sort_values('time')['factor'].values[:100])

### 因子层面：简单判断有效性
1. nan值
2. mean值

In [12]:
test_df = dl.df_dict['110053.csv'].copy()  # 用于测试函数
test_df = preprocess(test_df)
# 遍历判断哪些是直接可用的
current_factor_names = []  # 保存因子名称
indicators = []
for indicator_name in tqdm(valid_core_indicators):
    fun = getattr(test_df.ta, indicator_name)
    temp_data = fun(append=True)
    temp_factor_names = []
    if type(temp_data) == pd.core.series.Series:
        temp_factor_names = [temp_data.name]
    elif type(temp_data) == tuple:
        temp_factor_names = list(temp_data[0].columns)
    else:
        temp_factor_names = list(temp_data.columns)
    
    
    if 'order_book_id' in temp_factor_names:
        continue  # 说明这个不适合存储
    
    for _ in temp_factor_names:
        indicators.append(indicator_name)
    current_factor_names +=  temp_factor_names

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 407.64it/s]


In [13]:
# 构造一个df，用于特征名称和func之间的对应关系
func2factor_df = pd.DataFrame({
    'funcName':indicators,
    'factorName':current_factor_names,
})
print(func2factor_df.shape)

(158, 2)


### 清理明显不合适的特征

In [14]:
import cufflinks

In [15]:
factors_df = test_df[current_factor_names]

In [16]:
remove_cols = []
remove_cols += list(factors_df.columns[factors_df.isna().sum() > 200])  # nan值太多所以清理掉
remove_cols += list(factors_df.columns[factors_df.max() > 10000000])  # 均值过大所以清理掉
# 找到那些没有使用未来数据的因子: 最后一行必须有数值
invalid_cols = factors_df.columns[factors_df.isna().iloc[-1,:].values]
remove_cols += list(invalid_cols)
# remove_cols += ['EOM_14_100000000', 'VAR_30']

## 2. 构造cleaned factors函数

In [17]:
cleaned_factors = set(current_factor_names) - set(remove_cols)

In [18]:
len(current_factor_names)

158

In [19]:
len(cleaned_factors)

153

In [20]:
import importlib
# 使用 importlib.reload() 重新导入包
importlib.reload(factors)

<module 'factors' from '..\\factors.py'>

In [21]:
@factors.handle_list_data
def add_core_factors(raw_df:pd.DataFrame, core_indicators)-> pd.DataFrame:
    '''
    加入新因子的模板。
    :param raw_df: 原始数据
    :param core_indicators: 因子名称
    :return: 原始数据+因子
    '''
    raw_df = preprocess(raw_df)
    for indicator_name in valid_core_indicators:
        fun = getattr(raw_df.ta, indicator_name)
        temp_data = fun(append=True)
    return raw_df

In [22]:
for key, value in dl.df_dict.items():
    break

In [23]:
# 计算因子
add_core_factors(dl, core_indicators)

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10

F:\Pro

In [24]:
test_df = dl.df_dict['110053.csv'].copy()

# 3.查看因子IC

In [25]:
test_df.dropna(inplace=True)

In [26]:
record_dict = {}

for name, temp_df in dl.df_dict.items():
    record_dict[name] = temp_df[cleaned_factors].corrwith(temp_df['fwd_rtn_5_real'])

record_df = pd.DataFrame.from_records(record_dict)
mean_record = record_df.T.mean()
std_record = record_df.T.std()

In [31]:
mean_df = pd.DataFrame(mean_record)
mean_df.columns=['IC']
mean_df['IR'] = mean_record / std_record
mean_df['std'] = std_record
mean_df['abs_IR'] = np.abs(mean_df['IR'])

In [43]:
mean_df.sort_values('IR')[:10].index

Index(['INC_1', 'AOBV_LR_2', 'BOP', 'SMIo_5_20_5', 'STOCHRSIk_14_14_3_3',
       'TTM_TRND_6', 'CCI_14_0.015', 'J_9_3', 'BULLP_13', 'low_Z_30_1'],
      dtype='object')

In [52]:
mean_df.sort_values('abs_IR')[:10].index

Index(['UI_14', 'SQZPRO_ON_NARROW', 'TOS_STDEVALL_U_1', 'TOS_STDEVALL_L_1',
       'TOS_STDEVALL_U_2', 'TOS_STDEVALL_L_2', 'TOS_STDEVALL_L_3',
       'TOS_STDEVALL_U_3', 'TOS_STDEVALL_LR', 'SQZPRO_ON_WIDE'],
      dtype='object')

In [44]:
best_factors = ['INC_1', 'AOBV_LR_2', 'BOP', 'SMIo_5_20_5', 'STOCHRSIk_14_14_3_3',
       'TTM_TRND_6', 'CCI_14_0.015', 'J_9_3', 'BULLP_13', 'low_Z_30_1']

## 获取最佳特征和最佳ta functions

In [45]:
best_funcs = func2factor_df[func2factor_df['factorName'].isin(best_factors)]

In [46]:
best_funcs.reset_index(inplace=True, drop=True)

In [47]:
funcName = list(best_funcs.funcName.values)
factorName = list(best_funcs.factorName.values)

In [48]:
best_funcs

,funcName,factorName
0,eri,BULLP_13
1,bop,BOP
2,cci,CCI_14_0.015
3,increasing,INC_1
4,ttm_trend,TTM_TRND_6
5,smi,SMIo_5_20_5
6,cdl_z,low_Z_30_1
7,kdj,J_9_3
8,stochrsi,STOCHRSIk_14_14_3_3
9,aobv,AOBV_LR_2


In [51]:
# 2022的IR最高的因子组
print(funcName)
print(factorName)

['eri', 'bop', 'cci', 'increasing', 'ttm_trend', 'smi', 'cdl_z', 'kdj', 'stochrsi', 'aobv']
['BULLP_13', 'BOP', 'CCI_14_0.015', 'INC_1', 'TTM_TRND_6', 'SMIo_5_20_5', 'low_Z_30_1', 'J_9_3', 'STOCHRSIk_14_14_3_3', 'AOBV_LR_2']
